In [1]:
%pip install --upgrade pip
%pip install "snowflake-connector-python>=3,<4" python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.2 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.
  Using cached snowflake_connector_python-3.17.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (74 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached boto3-1.40.30-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.40.30-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
Using cached snowflake_connector_python-3.17.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.7 MB)
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 784.9 kB/s  0:00:05 eta 0:00:01
Using cached boto3-1.40.30-py3-none-any.whl (139 kB)
U

In [2]:
import snowflake.connector


In [3]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv, find_dotenv
import os

# Find and load the nearest .env (project root)
load_dotenv(find_dotenv(), override=True)

def need(name):
    v = os.getenv(name)
    if not v:
        raise RuntimeError(f"Missing required env var: {name}")
    # remove accidental quotes/spaces
    return v.strip().strip('"').strip("'")

SNOWFLAKE_USER     = need("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = need("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT  = need("SNOWFLAKE_ACCOUNT")  # e.g., afaypkh-rjb48354


In [5]:
import snowflake.connector

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
)

In [6]:
cs = conn.cursor()

In [7]:
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

In [8]:
cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

In [9]:
from pathlib import Path
import re

# ---- 0) connect  ----
conn = snowflake.connector.connect(
    host="qbmhuza-bnb86629.snowflakecomputing.com",   
    account="qbmhuza-bnb86629",                        
    user="second2",
    password="gyczeg6kaHqywownor",
    warehouse="COMPUTE_WH",                           
)
cs = conn.cursor()

# make sure the warehouse will wake itself
cs.execute("ALTER WAREHOUSE COMPUTE_WH SET AUTO_SUSPEND=60 AUTO_RESUME=TRUE")

# ---- 1) set DB/Schema (use your existing TESTDB) ----
DB, SCHEMA = "TESTDB", "PUBLIC"
cs.execute(f"CREATE DATABASE IF NOT EXISTS {DB}")
cs.execute(f"CREATE SCHEMA   IF NOT EXISTS {DB}.{SCHEMA}")
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SCHEMA}")

# ---- 2) read and run your .pgsql file from the repo ----
sql_path = Path("Data/supplier_case.pgsql")     
assert sql_path.exists(), f"Not found: {sql_path.resolve()}"
txt = sql_path.read_text(encoding="utf-8")

# tiny Postgres -> Snowflake cleanups
txt = "\n".join(l for l in txt.splitlines() if not l.strip().startswith("\\"))            # drop psql meta commands
txt = re.sub(r"\bNUMERIC\b", "NUMBER", txt, flags=re.I)                                   # NUMERIC -> NUMBER (safe)
txt = re.sub(r"\bsupplier_case\b", f"{DB}.{SCHEMA}.SUPPLIER_CASE", txt, flags=re.I)       # fully-qualify table

# split on semicolons and execute
stmts = [s.strip() for s in re.split(r";\s*(?=\n|$)", txt) if s.strip()]
for s in stmts:
    cs.execute(s)

# ---- 3) visualize (still only cs.execute) ----
print("Rows:", cs.execute(f"SELECT COUNT(*) FROM {DB}.{SCHEMA}.SUPPLIER_CASE").fetchone()[0])

print("\nSample rows:")
for r in cs.execute(f"""
    SELECT SupplierID, SupplierName, PhoneNumber, WebsiteURL,
           TRY_TO_DATE(ValidFrom) AS ValidFrom, TRY_TO_DATE(ValidTo) AS ValidTo
    FROM {DB}.{SCHEMA}.SUPPLIER_CASE
    ORDER BY SupplierID
    LIMIT 10
""").fetchall():
    print(r)

print("\nSchema:")
for r in cs.execute(f"DESCRIBE TABLE {DB}.{SCHEMA}.SUPPLIER_CASE").fetchall():
    print(r[0], r[1])


Rows: 13

Sample rows:
(1, 'A Datum Corporation', '(847) 555-0100', 'http://www.adatum.com', None, None)
(2, 'Contoso, Ltd.', '(360) 555-0100', 'http://www.contoso.com', None, None)
(3, 'Consolidated Messenger', '(415) 555-0100', 'http://www.consolidatedmessenger.com', None, None)
(4, 'Fabrikam, Inc.', '(203) 555-0104', 'http://www.fabrikam.com', None, None)
(5, 'Graphic Design Institute', '(406) 555-0105', 'http://www.graphicdesigninstitute.com', None, None)
(6, 'Humongous Insurance', '(423) 555-0105', 'http://www.humongousinsurance.com', None, None)
(7, 'Litware, Inc.', '(209) 555-0108', 'http://www.litwareinc.com', None, None)
(8, 'Lucerne Publishing', '(423) 555-0103', 'http://www.lucernepublishing.com', None, None)
(9, 'Nod Publishers', '(252) 555-0100', 'http://www.nodpublishers.com', None, None)
(10, 'Northwind Electric Cars', '(201) 555-0105', 'http://www.northwindelectriccars.com', None, None)

Schema:
SUPPLIERID NUMBER(38,0)
SUPPLIERNAME VARCHAR(16777216)
SUPPLIERCATEGORYID N

In [10]:
cs.execute("USE DATABASE TESTDB")
cs.execute("USE SCHEMA PUBLIC")

cs.execute("""
CREATE OR REPLACE TABLE TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN AS
SELECT
  CAST(SUPPLIERID               AS INT)        AS SUPPLIERID,
  SUPPLIERNAME                                   AS SUPPLIERNAME,
  CAST(SUPPLIERCATEGORYID       AS INT)        AS SUPPLIERCATEGORYID,
  CAST(PRIMARYCONTACTPERSONID   AS INT)        AS PRIMARYCONTACTPERSONID,
  CAST(ALTERNATECONTACTPERSONID AS INT)        AS ALTERNATECONTACTPERSONID,
  CAST(DELIVERYMETHODID         AS INT)        AS DELIVERYMETHODID,
  CAST(POSTALCITYID             AS INT)        AS POSTALCITYID,
  SUPPLIERREFERENCE                              AS SUPPLIERREFERENCE,
  PHONENUMBER                                   AS PHONENUMBER,
  WEBSITEURL                                    AS WEBSITEURL,
  DELIVERYADDRESSLINE1                           AS DELIVERYADDRESSLINE1,
  CAST(DELIVERYPOSTALCODE       AS INT)        AS DELIVERYPOSTALCODE,
  POSTALADDRESSLINE1                             AS POSTALADDRESSLINE1,
  CAST(POSTALPOSTALCODE         AS INT)        AS POSTALPOSTALCODE,
  CAST(LASTEDITEDBY             AS INT)        AS LASTEDITEDBY,
  TRY_TO_DATE(VALIDFROM)                        AS VALIDFROM,
  TRY_TO_DATE(VALIDTO)                          AS VALIDTO
FROM TESTDB.PUBLIC.SUPPLIER_CASE;
""")


In [11]:
print(cs.execute("SELECT COUNT(*) FROM TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN").fetchone()[0])
for r in cs.execute("""
  SELECT SUPPLIERID, SUPPLIERNAME, PHONENUMBER, WEBSITEURL, VALIDFROM, VALIDTO
  FROM TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN
  ORDER BY SUPPLIERID
  LIMIT 10
""").fetchall():
    print(r)


13
(1, 'A Datum Corporation', '(847) 555-0100', 'http://www.adatum.com', None, None)
(2, 'Contoso, Ltd.', '(360) 555-0100', 'http://www.contoso.com', None, None)
(3, 'Consolidated Messenger', '(415) 555-0100', 'http://www.consolidatedmessenger.com', None, None)
(4, 'Fabrikam, Inc.', '(203) 555-0104', 'http://www.fabrikam.com', None, None)
(5, 'Graphic Design Institute', '(406) 555-0105', 'http://www.graphicdesigninstitute.com', None, None)
(6, 'Humongous Insurance', '(423) 555-0105', 'http://www.humongousinsurance.com', None, None)
(7, 'Litware, Inc.', '(209) 555-0108', 'http://www.litwareinc.com', None, None)
(8, 'Lucerne Publishing', '(423) 555-0103', 'http://www.lucernepublishing.com', None, None)
(9, 'Nod Publishers', '(252) 555-0100', 'http://www.nodpublishers.com', None, None)
(10, 'Northwind Electric Cars', '(201) 555-0105', 'http://www.northwindelectriccars.com', None, None)


In [12]:
import snowflake.connector

def find_weather_db(cs):
    # Get all db names
    names = [r[1] for r in cs.execute("SHOW DATABASES").fetchall()]
    # Try exact names from the brief
    for cand in ("WEATHER__ENVIRONMENT", "WEATHER_ENVIRONMENT"):
        if cand in names:
            return cand
    # Fuzzy fallback (handles custom names)
    for n in names:
        if "WEATHER" in n and "ENVIRONMENT" in n:
            return n
    return None

def print_table_sample(cs, fqtn, sample_rows=5):
    print(f"\n=== {fqtn} ===")
    cs.execute(f"SELECT * FROM {fqtn} LIMIT {sample_rows}")
    rows = cs.fetchall()
    cols = [d[0] for d in cs.description]
    print("Columns:", ", ".join(cols))
    for i, r in enumerate(rows, 1):
        print(f"{i:>2}: {r}")
    cs.execute(f"SELECT COUNT(*) FROM {fqtn}")
    print("Total rows:", cs.fetchone()[0])

def print_cybersyn_weather_tables(conn, warehouse="COMPUTE_WH"):
    with conn.cursor() as cs:
        cs.execute(f"USE WAREHOUSE {warehouse}")

        db_name = find_weather_db(cs)
        if not db_name:
            print("⚠️  Skipping Cybersyn weather: no WEATHER…ENVIRONMENT database found in this account.")
            have = [r[1] for r in cs.execute("SHOW DATABASES").fetchall()]
            print("Databases you have:", have)
            return

        cs.execute(f"USE DATABASE {db_name}")

        # Prefer CYBERSYN schema if present; otherwise fall back to PUBLIC
        schemas = {r[1] for r in cs.execute(f"SHOW SCHEMAS IN DATABASE {db_name}").fetchall()}
        schema = "CYBERSYN" if "CYBERSYN" in schemas else "PUBLIC"
        cs.execute(f"USE SCHEMA {schema}")

        # If the exact NOAA table names differ, list what’s there and pick two NOAA* tables
        all_tables = [r[1] for r in cs.execute(f"SHOW TABLES IN SCHEMA {db_name}.{schema}").fetchall()]
        candidates = [t for t in all_tables if t.startswith("NOAA_")]
        if not candidates:
            print(f"No NOAA_* tables in {db_name}.{schema}. Available tables:", all_tables)
            return

        for t in candidates[:2]:
            print_table_sample(cs, f"{db_name}.{schema}.{t}", sample_rows=5)

In [13]:
import snowflake.connector

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
)
cs = conn.cursor()

In [14]:
import os, glob
from pathlib import Path

## Creating the PO_Table with Datatypes 
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SCHEMA}")
cs.execute(
"CREATE OR REPLACE TABLE PO_Data("
"purchaseorderid NUMBER(38,0), "
"supplierid NUMBER(38,0), "
"orderdate DATE, "
"deliverymethodid NUMBER(38,0), "
"contactpersonid NUMBER(38,0), "
"expecteddeliverydate DATE, "
"supplierreference VARCHAR, "
"isorderfinalized NUMBER(1,0), "
"comments VARCHAR, "
"internalcomments VARCHAR, "
"lasteditedby NUMBER(38,0), "
"purchaseorderlineid NUMBER(38,0), "
"stockitemid NUMBER(38,0), "
"orderedouters NUMBER(38,0), "
"description VARCHAR, "
"receivedouters NUMBER(38,0), "
"packagetypeid NUMBER(38,0), "
"expectedunitpriceperouter NUMBER(18,4), "
"lastreceiptdate DATE, "
"isorderlinefinalized NUMBER(1,0), "
"right_lasteditedby NUMBER(38,0), "
"right_lasteditedwhen TIMESTAMP_NTZ"
")")

# ---------- Resolve repo-relative data folder ----------
def find_monthly_po_dir() -> Path:
    """
    Locate the 'Data/Monthly PO Data' folder relative to the repository.
    Works from notebooks or scripts, on Windows/macOS/Linux.
    """
    candidates = [
        Path.cwd() / "Data" / "Monthly PO Data",
        Path.cwd() / "data" / "Monthly PO Data",
    ]

    # If running from a subfolder, search upward then rglob for the directory
    # 1) Walk up to (at most) 5 levels to find a '.git' folder (repo root)
    here = Path.cwd()
    ups = [here] + list(here.parents)[:5]
    repo_roots = [p for p in ups if (p / ".git").exists()]
    roots_to_search = repo_roots[:1] or [here]

    for root in roots_to_search:
        candidates.append(root / "Data" / "Monthly PO Data")
        candidates.append(root / "data" / "Monthly PO Data")
        # fallback: recursive search for the exact folder name
        for p in root.rglob("Monthly PO Data"):
            candidates.append(p)

    for p in candidates:
        if p.exists() and p.is_dir():
            # Must contain CSVs to be considered valid
            if any(p.glob("*.csv")):
                return p

    raise SystemExit("Could not find 'Data/Monthly PO Data' in this repo. "
                     "Make sure the data folder exists and contains .csv files.")

local_dir_path = find_monthly_po_dir()
local_dir = str(local_dir_path)  # keep your existing code style
print("Using data folder:", local_dir)

# ---------- Stage + file format ----------


cs.execute("CREATE OR REPLACE STAGE po_data_stage")
cs.execute("""
CREATE OR REPLACE FILE FORMAT po_csv_ff
  TYPE=CSV
  FIELD_DELIMITER=','
  FIELD_OPTIONALLY_ENCLOSED_BY='"'
  SKIP_HEADER=1
  TRIM_SPACE=TRUE
  EMPTY_FIELD_AS_NULL=TRUE
  NULL_IF=('','NULL','null','00:00.0','0:00.0','00:00','0:00')
  DATE_FORMAT='AUTO'
  TIME_FORMAT='AUTO'
  TIMESTAMP_FORMAT='AUTO'
""")

# ---------- Local files to stage (repo-relative) ----------

pattern = os.path.join(local_dir, "*.csv")
files = glob.glob(pattern)
print("Matched CSVs:", len(files))
if not files:
    raise SystemExit(f"No CSVs matched at: {pattern}")

# ---------- PUT files into stage (auto-compress -> .gz) ----------
for filepath in files:
    base = os.path.basename(filepath)
    if ":" in base:   # skip Windows ADS like ':Zone.Identifier'
        continue
    abs_path = os.path.abspath(filepath).replace("\\", "/")   # ensure forward slashes
    file_uri = "file:///" + abs_path.lstrip("/")              # exactly 3 slashes, no URL-encoding
    print("PUT ->", file_uri)
    cs.execute(f"PUT '{file_uri}' @po_data_stage AUTO_COMPRESS=TRUE OVERWRITE=TRUE")
    

# --- sanity check what's in the stage ---
cs.execute("LIST @po_data_stage")
print("Staged objects (top 10):", cs.fetchall()[:10])

# --- load into the table (skipping $12 = lasteditedwhen) ---
cs.execute("""
COPY INTO PO_Data
  FROM (
    SELECT
      $1  ::NUMBER(38,0)  AS purchaseorderid,
      $2  ::NUMBER(38,0)  AS supplierid,
      TRY_TO_DATE($3)     AS orderdate,
      $4  ::NUMBER(38,0)  AS deliverymethodid,
      $5  ::NUMBER(38,0)  AS contactpersonid,
      TRY_TO_DATE($6)     AS expecteddeliverydate,
      $7                  AS supplierreference,
      $8  ::NUMBER(1,0)   AS isorderfinalized,
      $9                  AS comments,
      $10                 AS internalcomments,
      $11 ::NUMBER(38,0)  AS lasteditedby,
      /* skip $12 */
      $13 ::NUMBER(38,0)  AS purchaseorderlineid,
      $14 ::NUMBER(38,0)  AS stockitemid,
      $15 ::NUMBER(38,0)  AS orderedouters,
      $16                 AS description,
      $17 ::NUMBER(38,0)  AS receivedouters,
      $18 ::NUMBER(38,0)  AS packagetypeid,
      $19 ::NUMBER(18,4)  AS expectedunitpriceperouter,
      TRY_TO_DATE($20)    AS lastreceiptdate,
      $21 ::NUMBER(1,0)   AS isorderlinefinalized,
      $22 ::NUMBER(38,0)  AS right_lasteditedby,
      TRY_TO_TIMESTAMP_NTZ($23) AS right_lasteditedwhen
    FROM @po_data_stage (FILE_FORMAT => 'po_csv_ff')
  )
  ON_ERROR = ABORT_STATEMENT
""")

Using data folder: /home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data
Matched CSVs: 41
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2019-6.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2021-10.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2020-1.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2022-3.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2021-1.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2020-8.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2021-7.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Dat

In [15]:
cs.execute("SELECT COUNT(*) FROM PO_Data")
print("Row count:", cs.fetchone()[0])
print("Row count should equal 8367")


Row count: 8367
Row count should equal 8367


In [16]:
cs.execute("SELECT * FROM PO_Data LIMIT 10")
for row in cs.fetchall():
    print(row)


(558, 4, datetime.date(2019, 12, 2), 7, 2, datetime.date(2019, 12, 22), '293092', 1, None, None, 8, 2160, 77, 767, '"The Gu" red shirt XML tag t-shirt (White) XXS', 767, 6, Decimal('84.0000'), datetime.date(2019, 12, 3), 1, 8, None)
(558, 4, datetime.date(2019, 12, 2), 7, 2, datetime.date(2019, 12, 22), '293092', 1, None, None, 8, 2161, 78, 981, '"The Gu" red shirt XML tag t-shirt (White) XS', 981, 6, Decimal('84.0000'), datetime.date(2019, 12, 3), 1, 8, None)
(558, 4, datetime.date(2019, 12, 2), 7, 2, datetime.date(2019, 12, 22), '293092', 1, None, None, 8, 2162, 80, 397, '"The Gu" red shirt XML tag t-shirt (White) M', 397, 6, Decimal('84.0000'), datetime.date(2019, 12, 3), 1, 8, None)
(558, 4, datetime.date(2019, 12, 2), 7, 2, datetime.date(2019, 12, 22), '293092', 1, None, None, 8, 2163, 86, 816, '"The Gu" red shirt XML tag t-shirt (White) 5XL', 816, 6, Decimal('96.0000'), datetime.date(2019, 12, 3), 1, 8, None)
(558, 4, datetime.date(2019, 12, 2), 7, 2, datetime.date(2019, 12, 22),

1) PO totals (POAmount) and a tidy PO header table

In [17]:
# One row per purchase order with the required total
cs.execute("""
CREATE OR REPLACE VIEW PO_Header AS
SELECT
  purchaseorderid,
  MIN(orderdate)                 AS orderdate,
  MIN(supplierid)                AS supplierid,
  SUM(receivedouters * expectedunitpriceperouter) AS POAmount
FROM PO_Data
GROUP BY purchaseorderid;
""")


In [35]:
from pathlib import Path
import shutil

# Path to the XML in your repo (adjust this if your folder name differs)
repo_xml = Path("Data") / "Supplier Transactions XML.xml"

# Make a simple, safe upload path that definitely exists *inside the container*
safe_dir = Path.home() / "sf_uploads"
safe_dir.mkdir(parents=True, exist_ok=True)

safe = safe_dir / "supplier_transactions.xml"   # normalize name
shutil.copy2(repo_xml, safe)                     # copy into place

uri = safe.as_uri()  # e.g., file:///home/jovyan/sf_uploads/supplier_transactions.xml
print("Local file URI:", uri)

# Make sure the stage exists and is an *internal* stage
cs.execute("CREATE STAGE IF NOT EXISTS invoice_xml_stage")

# Do the PUT
cs.execute(f"PUT '{uri}' @invoice_xml_stage OVERWRITE=TRUE AUTO_COMPRESS=TRUE")

# Confirm
print(cs.execute("LIST @invoice_xml_stage").fetchall())


Local file URI: file:///home/jovyan/sf_uploads/supplier_transactions.xml
[('invoice_xml_stage/supplier_transactions.xml.gz', 72528, '15c0f8f4276bf24ac457fb4e00e4107a', 'Sun, 14 Sep 2025 22:40:45 GMT')]


In [51]:
# 1) Recreate the XML file format WITH strip_outer_element
cs.execute("""
CREATE OR REPLACE FILE FORMAT xml_ff
  TYPE=XML
  STRIP_OUTER_ELEMENT=TRUE
""")
print("xml_ff (with STRIP_OUTER_ELEMENT=TRUE) ready.")


xml_ff (with STRIP_OUTER_ELEMENT=TRUE) ready.


In [52]:
# 2) Reload the XML so each <row> is its own table row
cs.execute("CREATE OR REPLACE TABLE INVOICE_RAW (doc VARIANT)")
cs.execute("""
COPY INTO INVOICE_RAW
FROM @invoice_xml_stage
FILE_FORMAT = xml_ff
ON_ERROR = ABORT_STATEMENT
""")
print("INVOICE_RAW rows (should be ~2438):", cs.execute("SELECT COUNT(*) FROM INVOICE_RAW").fetchone()[0])


INVOICE_RAW rows (should be ~2438): 2438


In [54]:
cs.execute("""
CREATE OR REPLACE TABLE SUPPLIER_INVOICES AS
SELECT
  TRY_TO_NUMBER(XMLGET(doc,'SupplierTransactionID'):"$"::STRING)      AS SupplierTransactionID,
  TRY_TO_NUMBER(XMLGET(doc,'SupplierID'):"$"::STRING)                  AS SupplierID,
  TRY_TO_NUMBER(XMLGET(doc,'TransactionTypeID'):"$"::STRING)           AS TransactionTypeID,
  NULLIF(XMLGET(doc,'PurchaseOrderID'):"$"::STRING,'')::NUMBER         AS PurchaseOrderID,
  NULLIF(XMLGET(doc,'SupplierInvoiceNumber'):"$"::STRING,'')           AS SupplierInvoiceNumber,
  TRY_TO_DATE(XMLGET(doc,'TransactionDate'):"$"::STRING)               AS TransactionDate,   -- safer
  TRY_TO_DECIMAL(XMLGET(doc,'AmountExcludingTax'):"$"::STRING,18,2)    AS AmountExcludingTax,
  TRY_TO_DECIMAL(XMLGET(doc,'TaxAmount'):"$"::STRING,18,2)             AS TaxAmount,
  TRY_TO_DECIMAL(XMLGET(doc,'TransactionAmount'):"$"::STRING,18,2)     AS TransactionAmount,
  TRY_TO_DECIMAL(XMLGET(doc,'OutstandingBalance'):"$"::STRING,18,2)    AS OutstandingBalance,
  TRY_TO_DATE(XMLGET(doc,'FinalizationDate'):"$"::STRING)              AS FinalizationDate,
  TRY_TO_BOOLEAN(XMLGET(doc,'IsFinalized'):"$"::STRING)                AS IsFinalized
FROM INVOICE_RAW
""")

print("Rows in SUPPLIER_INVOICES:", cs.execute("SELECT COUNT(*) FROM SUPPLIER_INVOICES").fetchone()[0])
print(cs.execute("SELECT * FROM SUPPLIER_INVOICES LIMIT 5").fetchall())


Rows in SUPPLIER_INVOICES: 2438
[(134, 2, 5, 1, '7290', datetime.date(2019, 1, 2), Decimal('313.50'), Decimal('47.03'), Decimal('360.53'), Decimal('0.00'), datetime.date(2019, 1, 7), True), (169, 4, 5, 2, '3898', datetime.date(2019, 1, 2), Decimal('21732.00'), Decimal('3259.80'), Decimal('24991.80'), Decimal('0.00'), datetime.date(2019, 1, 7), True), (186, 5, 5, 3, '616', datetime.date(2019, 1, 2), Decimal('2740.50'), Decimal('411.11'), Decimal('3151.61'), Decimal('0.00'), datetime.date(2019, 1, 7), True), (215, 7, 5, 4, '3869', datetime.date(2019, 1, 2), Decimal('42481.20'), Decimal('6372.19'), Decimal('48853.39'), Decimal('0.00'), datetime.date(2019, 1, 7), True), (224, 10, 5, 5, '4697', datetime.date(2019, 1, 2), Decimal('35067.50'), Decimal('5260.14'), Decimal('40327.64'), Decimal('0.00'), datetime.date(2019, 1, 7), True)]
